<a href="https://colab.research.google.com/github/ohilikeit/LG_Aimers_2023/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install optuna
! pip install catboost

In [27]:
import os
import warnings
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_squared_error

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier

from catboost import CatBoostRegressor
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

warnings.simplefilter(action='ignore', category=FutureWarning)
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)
os.chdir('/content/drive/MyDrive/ML_projects/LG_Aimers')
train = pd.read_csv('./train.csv').fillna(0)
test = pd.read_csv('./test.csv').fillna(0)
sample_submission = pd.read_csv('./sample_submission.csv')

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])
    
    for label in np.unique(test[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i])

# 값이 하나만 있는거 삭제(0으로 이미 채웠는데도 1개이면 모든 행이 동일한 값을 갖고있다는 뜻)
for i in train.columns:
    if len(train[i].value_counts().keys()) == 1:
        train.drop(i, axis=1, inplace=True)
        test.drop(i, axis=1, inplace=True)

X = train.drop(['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP'], axis=1)
y = train[['Y_Class', 'Y_Quality']]
test = test.drop(['PRODUCT_ID', 'TIMESTAMP'], axis=1)

# scaling
scaler = RobustScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
test = pd.DataFrame(scaler.transform(test), columns = test.columns)
test.drop(['LINE', 'PRODUCT_CODE'], axis=1, inplace=True)

train = pd.concat([X, y], axis=1)
train
# train.groupby(['LINE', 'PRODUCT_CODE'])['Y_Class'].describe()

,LINE,PRODUCT_CODE,X_1,X_2,X_4,X_5,X_7,X_8,X_9,X_10,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,Y_Class,Y_Quality
0,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.997188,1.000000,39.34,40.89,32.56,34.09,77.77,0.0,1,0.533433
1,-0.333333,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.995670,1.000000,38.89,42.82,43.92,35.34,72.55,0.0,2,0.541819
2,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.994547,1.000000,39.19,36.65,42.47,36.53,78.35,0.0,1,0.531267
3,-0.333333,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.999604,1.000000,37.74,39.17,52.17,30.58,71.78,0.0,2,0.537325
4,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.994631,0.997167,38.70,41.89,46.93,33.09,76.97,0.0,1,0.531590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.333333,0.0,0.0,0.042105,0.0,0.0,0.10,0.0,0.403846,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,1,0.526546
594,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,1.001290,1.000000,49.47,53.07,50.89,55.10,66.49,1.0,0,0.524022
595,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.993729,0.997167,0.00,0.00,0.00,0.00,0.00,1.0,0,0.521289
596,0.000000,-0.5,19.0,0.031579,0.0,0.1,0.00,0.0,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,1,0.531375


In [28]:
cutoff_df = train.groupby('Y_Class')['Y_Quality'].describe().reset_index()
cut_1 = (cutoff_df.loc[0, 'max'] + cutoff_df.loc[1, 'min']) / 2
cut_2 = (cutoff_df.loc[1, 'max'] + cutoff_df.loc[2, 'min']) / 2

train.drop('Y_Class', axis=1, inplace=True)

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 7)

# 학습함수 정의 
def objective(trial: Trial, train):
    param = {
        'verbose' : 0,
        'random_state': 7,
        'n_iter_no_change' : 50,
        'validation_fraction' : 0.1,
        'learning_rate' : 0.1,
        'n_estimators' : trial.suggest_int('n_estimators', 100, 2000, step=20),
        'min_impurity_decrease' : trial.suggest_float('min_impurity_decrease', 0, 0.2),
        'max_depth' : trial.suggest_int('max_depth', 2, 20),
        'max_features' : trial.suggest_categorical('max_features', ['auto', 'log2']),
        'subsample' : trial.suggest_float('subsample', 0.7, 1.0, step=0.1)
    }
    cv_accuracy = []
    cv = KFold(n_splits = 5)
    n_iter = 0

    for t,v in cv.split(train, train['Y_Quality']):
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        X = train_cv.drop('Y_Quality', axis=1)
        y = train_cv['Y_Quality']

        val_X = val_cv.drop('Y_Quality', axis=1)
        val_y = val_cv['Y_Quality']

        model = GradientBoostingRegressor(**param)

        model.fit(X, y)
        score = mean_squared_error(val_y, model.predict(val_X), squared = False)

        cv_accuracy.append(score)
        n_iter += 1

    return np.mean(cv_accuracy)

# create study
GB_study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='GB-Hyperparameter-Tuning'
)

# 학습 
GB_study.optimize(lambda trial: objective(trial,train), 
               n_trials=50)
print('Best trial : score {}, \nparams {}'.format(GB_study.best_trial.value, GB_study.best_trial.params))

In [ ]:
GB_model = GradientBoostingClassifier(**GB_study.best_trial.params, random_state = 42, n_iter_no_change = 50, validation_fraction = 0.1, learning_rate = 0.05)
GB_model.fit(X, y)
GB_pred = GB_model.predict(test)
print(pd.DataFrame(GB_pred).value_counts())

sample_submission['Y_Class'] = GB_pred
sample_submission.to_csv('./submit.csv', index=False)

In [ ]:
# SVC(kernel = 'poly)로 stacking이나 활용하자 / split은 0.2 기준 0 나왔음. 
# 할때 범주형 변수 labelencoding 해놨으니 얘네는 빼고 돌려보고 더 좋은 걸로 하자 

# y_class 나뉘는걸 보면 굉장히 민감하게 나뉘는거같은데 pca와 같은 차원축소 기법이 과연 옳을 것인가는 의문
# 이건 해보고 성능 높아지면 쓰고 아니면 안쓰기 

# 공정 데이터이고 각 컬럼이 비식별화되어 함부로 판단하기 힘듦. 
# 결측치로 비어있는 부분은 아마 line이나 product code가 다름에 따라 해당 공정이 없는 경우일 것으로 예상됨.
# 이상치 처리는 건들지 말고 결측치는 0으로 두어 모델에 반영되지 않게 하자. 

# line, product 4-1 이랑 5-1 조합 train에선 3 3 이고 test에선 3 1 인데 이거 빼고 아니고 성능 차이 확인 

## 차원 축소

In [7]:
from sklearn.decomposition import PCA
ex = X.iloc[:, :2]
only_X = X.iloc[:, 2:]
pca = PCA(n_components=10)
pca.fit(only_X)
X_pca = pca.transform(only_X)
print(X_pca.shape)
print(pca.explained_variance_ratio_)

df = pd.DataFrame(X_pca, columns = ['comp1', 'comp2'])
train = pd.concat([df, ex, y], axis=1)

(598, 10)
[6.44050778e-01 3.55949222e-01 3.86317897e-15 4.50832735e-16
 3.17290065e-18 1.61492020e-18 1.38025757e-18 4.02061723e-19
 1.90043129e-19 4.73762644e-20]


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
ex = X.iloc[:, :2]
only_X = X.iloc[:, 2:]

lda = LinearDiscriminantAnalysis()
lda.fit(only_X,y)
X_lda = lda.transform(only_X)


ex_2 = test.iloc[:, :2]
only_X_test = test.iloc[:, 2:]
test_lda = lda.transform(only_X_test)
test = pd.DataFrame(test_lda, columns = ['comp1', 'comp2'])
test = pd.concat([test, ex_2], axis=1)


df = pd.DataFrame(X_lda, columns = ['comp1', 'comp2'])
train = pd.concat([df, ex, y], axis=1)
X = train.drop('Y_Class', axis=1)
y = train['Y_Class']

## 1. XGB

In [ ]:
# 학습함수 정의 
def objective(trial: Trial, train):
    param = {
        'criterion' : 'absolute_error',
        'tree_method' : 'exact',
        'seed': 42, 
        'learning_rate' : trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators':trial.suggest_int('n_estimators', 100, 1000),        
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 9),
        'max_depth' : trial.suggest_int('max_depth', 3, 30),
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'min_split_loss' : trial.suggest_loguniform('min_split_loss', 1e-3, 10.0),
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
    }
    cv_accuracy = []
    cv = StratifiedKFold(n_splits = 5)
    n_iter = 0

    for t,v in cv.split(train, train['Y_Class']):
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        X = train_cv.drop('Y_Class', axis=1)
        y = train_cv['Y_Class']

        val_X = val_cv.drop('Y_Class', axis=1)
        val_y = val_cv['Y_Class']

        model = XGBClassifier(**param)

        model.fit(X, y)
        score = f1_score(val_y, model.predict(val_X), average = 'macro')

        cv_accuracy.append(score)
        n_iter += 1

    return np.mean(cv_accuracy)


# create study
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='XGBClassifier-Hyperparameter-Tuning'
)

# 학습 
study.optimize(lambda trial: objective(trial,train), 
               n_trials=50)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

## 2. LGBM

In [ ]:
# 학습함수 정의 
def objective(trial: Trial, train):

    param = {
        'verbose' : -1,
        'random_state': 7, 
        'n_jobs': -1,
        # 'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        # 'max_depth' : trial.suggest_int('max_depth', 2, 30),
        # 'n_estimators' : trial.suggest_int('n_estimators', 500, 3000),
        # 'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.3), 
        'learning_rate' : 0.1,
        # 'num_leaves' : trial.suggest_int('num_leaves', 5, 30),
        # 'min_child_samples' : trial.suggest_int('min_child_samples', 2, 10),
        # 'reg_alpha' : trial.suggest_float('reg_alpha', 1, 1.3),
        # 'reg_lambda' : trial.suggest_float('reg_lambda', 1, 1.3)
    }
    cv_accuracy = []
    cv = StratifiedKFold(n_splits = 5)
    n_iter = 0

    for t,v in cv.split(train, train['Y_Class']):
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        X = train_cv.drop('Y_Class', axis=1)
        y = train_cv['Y_Class']

        val_X = val_cv.drop('Y_Class', axis=1)
        val_y = val_cv['Y_Class']

        model = LGBMClassifier(**param)

        model.fit(X, y)
        score = f1_score(val_y, model.predict(val_X), average = 'macro')
        print(score)

        cv_accuracy.append(score)
        n_iter += 1

    return np.mean(cv_accuracy)

# create study
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='LGBM-Hyperparameter-Tuning'
)

# 학습 
study.optimize(lambda trial: objective(trial,train),
               n_trials=100)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, stratify = y, random_state = 7)

RF = RandomForestClassifier(random_state=7).fit(X_train, y_train)
RF_pred = RF.predict(X_val)
score = f1_score(y_val, RF_pred, average='macro')
print(score)

In [ ]:
model = LGBMClassifier(random_state=7).fit(X_train, y_train)
pred = model.predict(X_val)
score = f1_score(y_val, pred, average='macro')
print(score)

0.5929319467250501


In [ ]:
model = XGBClassifier(random_state=7).fit(X_train, y_train)
pred = model.predict(X_val)
score = f1_score(y_val, pred, average='macro')
print(score)

0.5925925925925926


In [ ]:
model = ExtraTreesClassifier(random_state=7).fit(X_train, y_train)
pred = model.predict(X_val)
score = f1_score(y_val, pred, average='macro')
print(score)

0.6415491578535056


In [ ]:
model = GradientBoostingClassifier(random_state=7).fit(X_train, y_train)
pred = model.predict(X_val)
score = f1_score(y_val, pred, average='macro')
print(score)

0.6888888888888888


In [ ]:
RF = RandomForestClassifier(random_state=43).fit(X, y)
RF_pred = RF.predict(test)

XGB = XGBClassifier(random_state=43).fit(X, y)
XGB_pred = XGB.predict(test)

LGBM = LGBMClassifier(random_state=43).fit(X, y)
LGBM_pred = LGBM.predict(test)

ET = ExtraTreesClassifier(random_state=43).fit(X, y)
ET_pred = ET.predict(test)

GB = GradientBoostingClassifier(random_state=43).fit(X, y)
GB_pred = GB.predict(test)

# 2랑 5가 멀쩡 

In [ ]:
# 5개 다 쓰는 hard voting
# https://wikidocs.net/42408

def mode(list):
    count = 0
    mode = 0;
    for x in list: 
        if list.count(x) > count:
            count = list.count(x)
            mode = x

    return mode
final_pred = []
for i in range(0, len(RF_pred)):
    final_pred.append(mode([RF_pred[i], XGB_pred[i], LGBM_pred[i], ET_pred[i], GB_pred[i]]))
sample_submission['Y_Class'] = final_pred
sample_submission['Y_Class'].value_counts()

1    268
0     40
2      2
Name: Y_Class, dtype: int64

In [ ]:
def mode(list):
    count = 0
    mode = 0;
    for x in list: 
        if list.count(x) > count:
            count = list.count(x)
            mode = x
    return mode
final_pred = []
for i in range(0, len(RF_pred)):
    final_pred.append(mode([GB_pred[i]]))
sample_submission['Y_Class'] = final_pred
sample_submission['Y_Class'].value_counts()

1    242
0     39
2     29
Name: Y_Class, dtype: int64

In [ ]:
sample_submission.to_csv('./submit.csv', index=False)

## 3. ET

In [ ]:
# 학습함수 정의 
def objective(trial: Trial, train):
    param = {
        'verbose' : 1,
        'random_state': 42,
        'n_jobs': -1,
        'max_depth' : trial.suggest_int('max_depth', 5, 50),
        'ccp_alpha' : trial.suggest_float('ccp_alpha', 0, 0.3),
        'n_estimators':trial.suggest_int('n_estimators', 500, 3000),
        'min_impurity_decrease' : trial.suggest_float('min_impurity_decrease', 0, 0.5)
    }
    cv_accuracy = []
    cv = StratifiedKFold(n_splits = 5)
    n_iter = 0

    for t,v in cv.split(train, train['Y_Class']):
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        X = train_cv.drop('Y_Class', axis=1)
        y = train_cv['Y_Class']

        val_X = val_cv.drop('Y_Class', axis=1)
        val_y = val_cv['Y_Class']

        model = ExtraTreesClassifier(**param)

        model.fit(X, y)
        score = f1_score(val_y, model.predict(val_X), average = 'macro')

        cv_accuracy.append(score)
        n_iter += 1

    return np.mean(cv_accuracy)


# create study
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='ExtraTrees-Hyperparameter-Tuning'
)

# 학습 
study.optimize(lambda trial: objective(trial,train), 
               n_trials=50)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

## 4. GB

In [24]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 7)

# 학습함수 정의 
def objective(trial: Trial, train):
    param = {
        'verbose' : 0,
        'random_state': 7,
        'n_iter_no_change' : 50,
        'validation_fraction' : 0.1,
        'learning_rate' : 0.1,
        'n_estimators' : trial.suggest_int('n_estimators', 100, 2000, step=20),
        'min_impurity_decrease' : trial.suggest_float('min_impurity_decrease', 0, 0.2),
        'max_depth' : trial.suggest_int('max_depth', 2, 20),
        'max_features' : trial.suggest_categorical('max_features', ['auto', 'log2']),
        'subsample' : trial.suggest_float('subsample', 0.7, 1.0, step=0.1)
    }
    cv_accuracy = []
    cv = KFold(n_splits = 5)
    n_iter = 0

    for t,v in cv.split(train, train['Y_Quality']):
        train_cv = train.iloc[t]
        val_cv = train.iloc[v]

        X = train_cv.drop('Y_Quality', axis=1)
        y = train_cv['Y_Quality']

        val_X = val_cv.drop('Y_Quality', axis=1)
        val_y = val_cv['Y_Quality']

        model = GradientBoostingRegressor(**param)

        model.fit(X, y)
        score = mean_squared_error(val_y, model.predict(val_X), squared = False)

        cv_accuracy.append(score)
        n_iter += 1

    return np.mean(cv_accuracy)

    # model = GradientBoostingRegressor(**param)
    # model.fit(X_train, y_train)
    # score = f1_score(y_val, model.predict(X_val), average='macro')

    # return score

# create study
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.HyperbandPruner(),
    study_name='GB-Hyperparameter-Tuning'
)

# 학습 
study.optimize(lambda trial: objective(trial,train), 
               n_trials=50)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

,LINE,PRODUCT_CODE,X_1,X_2,X_4,X_5,X_7,X_8,X_9,X_10,...,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,Y_Class,Y_Quality
0,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.997188,1.000000,39.34,40.89,32.56,34.09,77.77,0.0,1,0.533433
1,-0.333333,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.995670,1.000000,38.89,42.82,43.92,35.34,72.55,0.0,2,0.541819
2,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.994547,1.000000,39.19,36.65,42.47,36.53,78.35,0.0,1,0.531267
3,-0.333333,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.999604,1.000000,37.74,39.17,52.17,30.58,71.78,0.0,2,0.537325
4,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.994631,0.997167,38.70,41.89,46.93,33.09,76.97,0.0,1,0.531590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.333333,0.0,0.0,0.042105,0.0,0.0,0.10,0.0,0.403846,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,1,0.526546
594,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,1.001290,1.000000,49.47,53.07,50.89,55.10,66.49,1.0,0,0.524022
595,-0.666667,-1.0,-1.0,-0.957895,-1.0,-1.0,-0.90,-1.0,-0.596154,-1.0,...,0.993729,0.997167,0.00,0.00,0.00,0.00,0.00,1.0,0,0.521289
596,0.000000,-0.5,19.0,0.031579,0.0,0.1,0.00,0.0,0.000000,0.0,...,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,1,0.531375


In [ ]:
GB_model = GradientBoostingClassifier(random_state = 42, n_iter_no_change = 50, validation_fraction = 0.1, n_estimators = 1900, 
                                      learning_rate = 0.05, min_impurity_decrease = 0.07031308605404483, max_depth = 2)
GB_model.fit(X, y)
GB_pred = GB_model.predict(test)
pd.DataFrame(GB_pred).value_counts()

1    242
2     39
0     29
dtype: int64

In [ ]:
GB_model = GradientBoostingClassifier(**study.best_trial.params, random_state = 42, n_iter_no_change = 50, validation_fraction = 0.1, learning_rate = 0.05)
GB_model.fit(X, y)
GB_pred = GB_model.predict(test)
print(pd.DataFrame(GB_pred).value_counts())

sample_submission['Y_Class'] = GB_pred
sample_submission.to_csv('./submit.csv', index=False)

1    241
2     39
0     30
dtype: int64
